inner buffer - mosaic - filter

In [5]:
#@title Import

import os
import sys

# change system patch to be able to access the custom modules (functions, normalization)
# src: https://colab.research.google.com/drive/1uvHuizCBqFgvbCwEhK7FvU8JW0AfxgJw#scrollTo=psH0aLrvoh78
module_path = os.path.abspath(os.path.join('/home/lucie/GDA_UGS/ESA_GDA_UGS/GDA_urban'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import glob

import csv

from osgeo import osr, gdal
import json

import keras
from keras import utils
from keras.applications import VGG16
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, Concatenate, BatchNormalization, Activation, Dropout
from keras import metrics
from keras.models import Model
from keras.callbacks import EarlyStopping

import tensorflow as tf

#from PIL import Image
from matplotlib import pyplot as plt

#import functions
import functions_predict
import normalization

from pathlib import Path

import subprocess

#from sklearn.metrics import classification_report, confusion_matrix

In [9]:
#@title Parameters
#size = 224
size = 448

#class_codes = list(range(1,9)) #NBS Kinshasa
class_codes = list(range(0,8)) #Amman P2

base_dir=Path('/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa')
#base_dir=Path('/home/lucie/GDA_UGS/ESA_GDA_UGS/GDA_urban/Phase2')

version='scenario_06'
subversion='1'
#version='01'
#subversion='7'

#work_dir = base_dir.joinpath('02_model_input','data_predict_outsideAOI_overlap25_v2')
#work_dir = base_dir.joinpath('testing',f'version_{version}_{subversion}','predict_outsideAOI_out')

model_output_parent_folder = base_dir.joinpath('04_prediction',f'model_{version}_{subversion}')
#model_output_parent_folder = base_dir.joinpath('testing',f'version_{version}_{subversion}','predict_outsideAOI_out_v4')
work_dir = model_output_parent_folder

band_list= ['b01', 'b02', 'b03']

#input_rasters_folder = Path('/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/predict_test_scenario_06/img_clip_test_python')
#output_rasters_folder = Path('/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/predict_test_scenario_06/img_clip_test_python_output')

# INNER BUFFER params
buffer_output_parent_folder = base_dir.joinpath('04_prediction',f'model_{version}_{subversion}','01_inner_buffer')
#buffer_output_parent_folder = base_dir.joinpath('testing',f'version_{version}_{subversion}','01_inner_buffer')
nodata_value = -32678
overlap = .25
epsg = '32733' #Kinshasa
#epsg = '32636' #Amman

# MOSAIC PARAMS
mosaic_input_parent_folder = model_output_parent_folder
mosaic_output_parent_folder = base_dir.joinpath('04_prediction',f'model_{version}_{subversion}','02_mosaic')
#mosaic_output_parent_folder = base_dir.joinpath('testing',f'version_{version}_{subversion}','02_mosaic')

#FILTERING
threshold = '8' #number of pixels to be removed
filter_output_parent_folder = base_dir.joinpath('04_prediction',f'model_{version}_{subversion}','03_filter')
#filter_output_parent_folder = base_dir.joinpath('testing',f'version_{version}_{subversion}','03_filter')


In [10]:
def read_raster_info(raster_filepath):
    ds = gdal.Open(str(raster_filepath))

    RasterXSize = ds.RasterXSize
    RasterYSize = ds.RasterYSize
    gt = ds.GetGeoTransform()

    xmin = gt[0]
    ymax = gt[3]
    pixel_width = gt[1]
    yres = gt[5]

    ds = None
    return (xmin, ymax, RasterXSize, RasterYSize, pixel_width)


def inner_buffer(input_rasters_folder,output_rasters_folder,epsg):
    
    raster_list = os.listdir(input_rasters_folder)
    output_raster_filepath_list = []

    for raster_list_item in raster_list:

            input_raster = input_rasters_folder.joinpath(raster_list_item)
            output_raster = output_rasters_folder.joinpath(raster_list_item)


            if not input_raster.exists():
                raise Exception("here")

            (xmin, ymax, aoi_width_pixels, aoi_height_pixels, pixel_width) = read_raster_info(input_raster)


            x_offset = overlap/2*aoi_width_pixels
            y_offset = overlap*aoi_height_pixels/2
            tile_width = (1-overlap)*aoi_width_pixels
            tile_height =  (1-overlap)*aoi_width_pixels

            tile_ulx = xmin + x_offset * pixel_width
            tile_uly = ymax - y_offset * pixel_width
            tile_lrx = tile_ulx + tile_width * pixel_width
            tile_lry = tile_uly - tile_height * pixel_width

            args = ["gdalwarp",
                   "-t_srs", "EPSG:{}".format(epsg),
                   "-te", str(tile_ulx), str(tile_lry), str(tile_lrx),
                   str(tile_uly),
                   "-r", 'near',
                   "-co", "COMPRESS=DEFLATE",
                   str(input_raster),
                   str(output_raster)]
            cmd_output = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print("exit code {} --> {}".format(cmd_output, args))

            output_raster_filepath_list.append(output_raster)
    return output_raster_filepath_list

def mosaic_tifs_in_folder(mosaic_input_folder, mosaic_filepath):
    
    raster_list = os.listdir(mosaic_input_folder)
    print(f"raster list {raster_list}")
    
    input_raster_filepath_list = []
    for raster_list_item in raster_list:
            input_raster_filepath_list.append(mosaic_input_folder.joinpath(raster_list_item))
            
    print(input_raster_filepath_list)
    
    input_raster_filepath_list_v2=[str(fp) for fp in input_raster_filepath_list]
            
    if Path(mosaic_filepath).is_file():
        print('***previous version deleted***')
        Path(mosaic_filepath).unlink()
    
    args = ["gdal_merge.py",
    "-co", "TILED=YES",
    "-co", "COMPRESS=DEFLATE",
    "-n", "0",
    "-o", str(mosaic_filepath),
    "-pct"] + [str(fp) for fp in input_raster_filepath_list_v2]

    cmd_output =subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(f"exit code {cmd_output} --> {args}")
    
def sieve_filter(threshold,filter_input_filepath,filter_output_filepath):
    gdal_cmd = ["gdal_sieve.py", "-st", threshold, "-8", "-nomask", filter_input_filepath, "-of", "GTIFF", filter_output_filepath]
    subprocess.run(gdal_cmd)
    

In [11]:
## DATA - set paths HERE

os.makedirs(model_output_parent_folder, exist_ok=True)

"""
if os.path.exists(model_output_parent_folder) == False:
    os.mkdir(model_output_parent_folder)
"""

#list_cells=['36','38','9']
#list_cells=os.listdir('/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_test')
list_cells=os.listdir(work_dir)

for part in list_cells:
    print(part)

    #inner buffer
    print('inner buffer...')
    buffer_input_subfolder = mosaic_input_parent_folder.joinpath(part)
    buffer_output_subfolder = buffer_output_parent_folder.joinpath(part)
    os.makedirs(buffer_output_subfolder, exist_ok=True)

    inner_buffer(buffer_input_subfolder,buffer_output_subfolder,epsg)

    #mosaic
    print('mosaic...')
    mosaic_input_folder = buffer_output_subfolder

    mosaic_output_filepath = mosaic_output_parent_folder.joinpath(f'mosaic_{part}.tif')
    os.makedirs(mosaic_output_parent_folder, exist_ok=True)

    print("generating "+str(mosaic_output_filepath))

    mosaic_tifs_in_folder(mosaic_input_folder, mosaic_output_filepath)

    #filtering
    print('filtering...')
    filter_input_filepath=mosaic_output_filepath
    filter_output_filepath = filter_output_parent_folder.joinpath(f'filtered_{part}.tif')
    os.makedirs(filter_output_filepath.parent, exist_ok=True)

    sieve_filter(threshold,filter_input_filepath,filter_output_filepath)
    #gdal_cmd = ["gdal_sieve.py", "-st", threshold, "-8", "-nomask", filter_input_filepath, "-of", "GTIFF", filter_output_filepath]
    #subprocess.run(gdal_cmd)

    print('')
        

img_tile_310
inner buffer...
exit code CompletedProcess(args=['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '530898.1756', '9515459.5087', '531066.1756', '9515627.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0001_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_310/0001_img.tif'], returncode=0, stdout=b'Creating output file that is 336P x 336L.\nProcessing /home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0001_img.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.\n', stderr=b'') --> ['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '530898.1756', '9515459.5087', '531066.1756', '9515627.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0001_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_pred

exit code CompletedProcess(args=['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '531234.1756', '9515291.5087', '531402.1756', '9515459.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0010_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_310/0010_img.tif'], returncode=0, stdout=b'Creating output file that is 336P x 336L.\nProcessing /home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0010_img.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.\n', stderr=b'') --> ['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '531234.1756', '9515291.5087', '531402.1756', '9515459.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0010_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01

exit code CompletedProcess(args=['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '531402.1756', '9515123.5087', '531570.1756', '9515291.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0018_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_310/0018_img.tif'], returncode=0, stdout=b'Creating output file that is 336P x 336L.\nProcessing /home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0018_img.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.\n', stderr=b'') --> ['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '531402.1756', '9515123.5087', '531570.1756', '9515291.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0018_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01

exit code CompletedProcess(args=['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '531570.1756', '9514955.5087', '531738.1756', '9515123.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0026_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_310/0026_img.tif'], returncode=0, stdout=b'Creating output file that is 336P x 336L.\nProcessing /home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0026_img.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.\n', stderr=b'') --> ['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '531570.1756', '9514955.5087', '531738.1756', '9515123.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0026_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01

exit code CompletedProcess(args=['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '531738.1756', '9514787.5087', '531906.1756', '9514955.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0034_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_310/0034_img.tif'], returncode=0, stdout=b'Creating output file that is 336P x 336L.\nProcessing /home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0034_img.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.\n', stderr=b'') --> ['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '531738.1756', '9514787.5087', '531906.1756', '9514955.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310/0034_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01

...

exit code CompletedProcess(args=['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '545010.1756', '9506947.5087', '545178.1756', '9507115.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_617/0033_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_617/0033_img.tif'], returncode=0, stdout=b'Creating output file that is 336P x 336L.\nProcessing /home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_617/0033_img.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.\n', stderr=b'') --> ['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '545010.1756', '9506947.5087', '545178.1756', '9507115.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_617/0033_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01

exit code CompletedProcess(args=['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '545178.1756', '9506779.5087', '545346.1756', '9506947.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_617/0041_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_617/0041_img.tif'], returncode=0, stdout=b'Creating output file that is 336P x 336L.\nProcessing /home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_617/0041_img.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.\n', stderr=b'') --> ['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '545178.1756', '9506779.5087', '545346.1756', '9506947.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_617/0041_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01

exit code CompletedProcess(args=['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '545346.1756', '9506611.5087', '545514.1756', '9506779.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_617/0049_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_617/0049_img.tif'], returncode=0, stdout=b'Creating output file that is 336P x 336L.\nProcessing /home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_617/0049_img.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.\n', stderr=b'') --> ['gdalwarp', '-t_srs', 'EPSG:32733', '-te', '545346.1756', '9506611.5087', '545514.1756', '9506779.5087', '-r', 'near', '-co', 'COMPRESS=DEFLATE', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_617/0049_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01

exit code CompletedProcess(args=['gdal_merge.py', '-co', 'TILED=YES', '-co', 'COMPRESS=DEFLATE', '-n', '0', '-o', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/02_mosaic/mosaic_img_tile_617.tif', '-pct', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_617/0001_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_617/0002_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_617/0003_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_617/0004_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_617/0005_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/01_inner_buffer/img_tile_617/0006_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_p

0...10...20...30...40...50...60...70...80...90...100 - done.

